<a href="https://colab.research.google.com/github/dhawalnegi1/EVA/blob/master/Project-6/Assignment6_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The Model using function api having normal, spatial sperable, depthwise seperable and grouped convolution.

Import all requirements(Input, merge, SeperableConv2D, Concatenate are new inclusions)

In [1]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2017)

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation,SeparableConv2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.layers import Reshape,Input,Lambda
from keras.layers.merge import concatenate

Using TensorFlow backend.


Import the cifar dataset and store in the variables for training and test purpose.

In [2]:
# Load CIFAR10 Data
from keras.datasets import cifar10
(xtrain, ytrain), (xtest, ytest) = cifar10.load_data()
img_height, img_width, channel = xtrain.shape[1],xtrain.shape[2],xtrain.shape[3]
num_classes = len(np.unique(ytest))
print(num_classes)

10


Function accuracy is defined to calculate the validation accuracy by finding total no of images classfied correctly

In [0]:
def accuracy(xtest, ytest, model):
    result = model.predict(xtest)
    predicted_class = np.argmax(result, axis=1)
    true_class = np.argmax(ytest, axis=1)
    num_correct = np.sum(predicted_class == true_class) 
    accuracy = float(num_correct)/result.shape[0]
    return (accuracy * 100)

Pixel Normalisation is done for training and testing data. And the class labels for training and testing data is converted into one-hot encoding.

In [0]:
xtrain = xtrain.astype('float32')/255
xtest = xtest.astype('float32')/255
# convert class labels to binary class labels
ytrain = np_utils.to_categorical(ytrain, num_classes)
ytest = np_utils.to_categorical(ytest, num_classes)

##Block1 - Normal Convolution

2 layers of normal 3x3 convolution is add and after that bottleneck layer of 1x1 and maxpooling is used.

In [5]:
input = Input(shape=(img_height, img_width, channel,))

# BLOCK 1
L11 = Conv2D(32, (3,3), strides=(1,1), name='normalconv1', use_bias=False,padding='same')(input)
L11 = BatchNormalization()(L11)
L11 = Activation('relu')(L11)

L12 = Conv2D(64, (3,3), strides=(1,1), name='normalconv2', use_bias=False)(L11)
L12 = BatchNormalization()(L12)
L12 = Activation('relu')(L12)

L13 = Conv2D(16,1,use_bias=False)(L12)
L13 = MaxPooling2D(pool_size=(2, 2))(L13)

Instructions for updating:
Colocations handled automatically by placer.


##Block2 - Spatoally Seperable Convolution
2 layers of Spatially Separable Convolution(3x1 and 1x3) is add and after that bottleneck layer of 1x1 and maxpooling is used.

In [0]:
#BLOCK 2
L21 = Conv2D(16, (3,1), strides=(1,1), name='spatialseperable1-3-1', use_bias=False)(L13)
L21 = BatchNormalization()(L21)
L21 = Activation('relu')(L21)
L21 = Conv2D(32, (1,3), strides=(1,1), name='spatialseperable1-1-3', use_bias=False)(L21)
L21 = BatchNormalization()(L21)
L21 = Activation('relu')(L21)

L22 = Conv2D(32, (3,1), strides=(1,1), name='spatialseperable2-3-1', use_bias=False)(L21)
L22 = BatchNormalization()(L22)
L22 = Activation('relu')(L22)
L22 = Conv2D(64, (1,3), strides=(1,1), name='spatialseperable2-1-3', use_bias=False)(L22)
L22 = BatchNormalization()(L22)
L22 = Activation('relu')(L22)

L23 = Conv2D(32,1,use_bias=False)(L22)
L23 = MaxPooling2D(pool_size=(2, 2))(L23)

#Block3 - Depthwise Separable Convolution
1 Depthwise Separable Convolution layer is apllied which first seperate all the input channel and then apply 3x3 on each individually and then 1x1 is applied to combine them.

In [0]:
L3 = SeparableConv2D(64, (3,3), strides=(1, 1), depth_multiplier=1, use_bias=False, name='depthwise-seperable')(L23)
L3 = BatchNormalization()(L3)
L3 = Activation('relu')(L3)

#Block4 - Grouped Convolution (use 3x3, 5x5 only)
The group convolution means 2 diffent set of convolution is applied on input layer in parallel.
3x3 is applied on L3. and parallely 2 3x3 are applied , and then output of both convolution are concatneted.

In [0]:
L41 = Conv2D(128, (3,3), strides=(1,1), name='3x3', use_bias=False)(L3)
L41 = BatchNormalization()(L41)
L41 = Activation('relu')(L41)

L42 = Conv2D(64, (3,3), strides=(1,1), name='5x5-1',padding='same', use_bias=False)(L3)
L42 = BatchNormalization()(L42)
L42 = Activation('relu')(L42)
L42 = Conv2D(128, (3,3), strides=(1,1), name='5x5-2', use_bias=False)(L42)
L42 = BatchNormalization()(L42)
L42 = Activation('relu')(L42)

L4 = concatenate([L41,L42])

L4 = Conv2D(32,1)(L4)

#Block5 - Grouped Convolution (3x3 with dilation of 1 and 2)
The group convolution means 2 diffent set of convolution is applied on input layer in parallel.
3x3 with dilation 1 is applied on L4. and parallely 3x3 with dilation of 2 is applied , and then output of both convolution are concatneted.

In [0]:
L51 = Conv2D(64, (3,3), strides=(1,1), name='dilation-1',padding='same', use_bias=False,dilation_rate=(1, 1))(L4)
L51 = BatchNormalization()(L51)
L51 = Activation('relu')(L51)

L52 = Conv2D(64, (3,3), strides=(1,1), name='dilation-2',padding='same', use_bias=False,dilation_rate=(2, 2))(L4)
L52 = BatchNormalization()(L52)
L52 = Activation('relu')(L52)

L5 = concatenate([L51,L52])

Finally the no of channels are reduced to 10 and flatten and softmax is applied.

In [0]:
OUTPUT = Conv2D(10,1)(L5)
OUTPUT = Flatten()(OUTPUT)
Output = Activation('softmax')(OUTPUT)

In [0]:
# Compile the model
model= Model(inputs=[input],outputs=[Output])


Model is compiled and summery is printed.

In [12]:
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
normalconv1 (Conv2D)            (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         normalconv1[0][0]                
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
normalconv

Model is Trained for 50 epocs and checkpoint is used to saved the model with best validation accuracy and time taken to run is calculated. 

In [13]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=False)

from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
# train the model
start = time.time()
# Train the model
filepath="saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model_info = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size = 128),
                                 samples_per_epoch = xtrain.shape[0], nb_epoch = 50, 
                                 validation_data = (xtest,ytest),callbacks= callbacks_list, verbose=1)
end = time.time()
print ("Model took %0.2f seconds to train"%(end - start))

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., verbose=1, steps_per_epoch=390, epochs=50)`


Epoch 1/50
390/390 [==============================] - 13s 33ms/step - loss: 1.5521 - acc: 0.4310 - val_loss: 1.7086 - val_acc: 0.4227

Epoch 00001: val_acc improved from -inf to 0.42270, saving model to saved.hdf5
Epoch 2/50
390/390 [==============================] - 10s 24ms/step - loss: 1.2110 - acc: 0.5651 - val_loss: 1.6198 - val_acc: 0.4390

Epoch 00002: val_acc improved from 0.42270 to 0.43900, saving model to saved.hdf5
Epoch 3/50
390/390 [==============================] - 9s 24ms/step - loss: 1.0424 - acc: 0.6274 - val_loss: 1.1313 - val_acc: 0.5974

Epoch 00003: val_acc improved from 0.43900 to 0.59740, saving model to saved.hdf5
Epoch 4/50
390/390 [==============================] - 9s 24ms/step - loss: 0.9287 - acc: 0.6672 - val_loss: 1.1201 - val_acc: 0.6072

Epoch 00004: val_acc improved from 0.59740 to 0.60720, saving model to saved.hdf5
Epoch 5/50
390/390 [==============================] - 9s 24ms/step - loss: 0.8439 - acc: 0.6980 - val_loss: 0.9871 - val_acc: 0.6469

Epo

The best model is printed and accuracy is printed.

In [14]:
from keras.models import load_model
model = load_model('saved.hdf5')
score = model.evaluate(xtest, ytest, verbose=1)
print(score)

10000/10000 [==============================] - 2s 153us/step
[1.5031730436325073, 0.7005]
